In [ ]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt

In [ ]:
# Set up paths

path_to_data = '/expanse/nfs/cw3e/cwp140/'     # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
temporal_res = '6hr'
community_lst = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
varname = 'PCPT' # 'PCPT' or 'UV'

fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format(varname)
wrf = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')
if temporal_res == '6hr':
    wrf = wrf.resample(time="6H").sum('time') # resample WRF data to be mm per day
elif (temporal_res == 'daily') & (varname == 'PCPT'):
    wrf = wrf.resample(time="1D").sum('time') # resample WRF data to be mm per day
    
elif (temporal_res == 'daily') & (varname == 'UV'):
    wrf = wrf.sel(lev='1000')
    wrf = wrf.resample(time="1D").mean('time') # resample WRF data to be m s-1

wrf

In [ ]:
%%time
## need to rechunk so time is a single chunk
wrf = wrf.chunk(dict(time=-1))

quantile_arr = np.array([0, .75, .95])

## Calculate the percentiles
quantile = wrf.quantile(quantile_arr, dim=['time'], skipna=True)

In [ ]:
quantile

In [ ]:
%%time
# write to netCDF
fname = os.path.join(path_to_data, 'preprocessed/SEAK-WRF-PCPT/PCPT_quantiles_{0}.nc'.format(temporal_res))
quantile.load().to_netcdf(path=fname, mode = 'w', format='NETCDF4')